<a href="https://colab.research.google.com/github/JunetzMasihBelajar/2025_ComputerVision_TI2B/blob/main/jobsheet3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>